# QNN4EO: a quantum convolutional neural network for satellite data classification

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

## Load the dataset

The dataset is loaded through the DatasetHandler class.

In [ ]:
from DatasetHandler import DatasetHandler

### 1. Initialize the dataset handler

The dataset handler is initialized using the root path of the dataset. In this cases the root path is  *EuroSAT*, this folder contains a sub-folder for each class as printed below.

In [ ]:
dataset_root_path = 'EuroSAT'
datasetHandler = DatasetHandler(dataset_root_path)
datasetHandler.print_classes()

### 2. Select the two classes for the binary classification
In order to run the binary classification, the used should select two classes from the dataset

In [ ]:
class_1 = int(input('Class 1 [0-9]: '))
print('\t\t',datasetHandler.classes[class_1], '\n')

class_2 = int(input('Class 2 [0-9]: '))
print('\t\t',datasetHandler.classes[class_2], '\n')

classes_name = []
classes_name.append(datasetHandler.classes[class_1].split('/')[-1])
classes_name.append(datasetHandler.classes[class_2].split('/')[-1])

### 3. Load paths and labels
After the selection of the two classes, the Dataset Handler loads images paths and images labels for both the two classes

In [ ]:
imgs_path, imgs_label = datasetHandler.load_paths_labels(classes = [datasetHandler.classes[class_1], datasetHandler.classes[class_2]])

print('Dataset images: ', len(imgs_path))
print('Dataset labels: ', len(imgs_label))
print('Dataset sample -> image path: ', imgs_path[0], ' image label', imgs_label[0])

### 4. Trainining-Validation split
The paths and labels are then divided randomly in training and validation datasets

In [ ]:
train_images, train_labels, val_images, val_labels = datasetHandler.train_validation_split(imgs_path, imgs_label, split_factor = 0.2)
print('Training images: ',  train_images.shape)
print('Training labels: ',  train_labels.shape)
print('Validatiom images: ',  val_images.shape)
print('Validation labels: ',  val_labels.shape)

### 5. Display a sample of the dataset
In order to visualize the images and lables of the two classes selected, the utility *plotDataset* can be used to plot a sample of the training/validation dataset.

In [ ]:
from utils import plotDataset
plotDataset(train_images, train_labels, classes_name, columns = 5, rows = 2)

## CNN classifier

In [ ]:
from CNN_Classifier import CNN_Classifier

### 1. Initialize the CNN model

In [ ]:
img_shape = plt.imread(train_images[0]).shape
n_classes = train_labels.shape[1]

cnn = CNN_Classifier(img_shape = img_shape, n_classes = n_classes)

### 2. Train the CNN model

In [ ]:
batch_size = 1
epochs = 20
train_generator = datasetHandler.cnn_data_loader(train_images, train_labels, batch_size = batch_size, img_shape = img_shape, n_classes = n_classes)
val_generator = datasetHandler.cnn_data_loader(val_images, val_labels, batch_size = batch_size, img_shape = img_shape, n_classes = n_classes)

history = cnn.train_model(epochs, batch_size, train_generator, val_generator, len(train_images), len(val_images))

#### After training the CNN model, save it in the trained_models folder

In [ ]:
cnn.model.save(os.path.join('trained_models', 'CNN-' + classes_name[0] + '-VS-' + classes_name[1] + '.h5'))

### 3. Test the CNN model

In [ ]:
from tensorflow.keras.models import load_model
cnn.model = load_model(os.path.join('trained_models', 'CNN-' + classes_name[0] + '-VS-' + classes_name[1] + '.h5'))

In [ ]:
val_data_generator = datasetHandler.cnn_data_loader(val_images, val_labels, batch_size = len(val_images), img_shape = img_shape, n_classes = n_classes)
accuracy, precision, recall, f1 = cnn.evaluate_model(val_data_generator, classes_name)

## QNN4EO

In [ ]:
from QNN4EO import QNN4EO
import torch

### 1. Initialize the model

In [ ]:
qnn4eo = QNN4EO()

### 2. Traing the QNN4EO model

In [ ]:
img_shape = plt.imread(train_images[0]).shape
n_classes = train_labels.shape[1]
epochs = 20

train_generator = datasetHandler.qcnn_data_loader(train_images, train_labels, batch_size = 1, img_shape = img_shape)
history = qnn4eo.train_model(epochs, train_generator, len(train_images))

In [ ]:
torch.save(qnn4eo.state_dict(), os.path.join('trained_models', 'QNN4EO-' + classes_name[0] + '-VS-' + classes_name[1] + '.h5'))

### 3. Test the QNN4EO model

In [ ]:
qnn4eo.load_state_dict(torch.load(os.path.join('trained_models', 'QNN4EO-' + classes_name[0] + '-VS-' + classes_name[1] + '.h5')))

In [ ]:
val_data_gen = datasetHandler.qcnn_data_loader(val_images, val_labels, batch_size = 1, img_shape = img_shape)
accuracy, precision, recall, f1 = qnn4eo.evaluate_model(val_data_gen, len(val_images))

# Running all the binary classification tests
### Keep Attention! This process can last a lot

The next cells can be used to automatically run all the possible binary classification test.

For this cases, since the EuroSAT dataset contains 10 classes, the script will run 45 binary classifications. Thi script is structured as follow
- Loading datates
- CNN training
- CNN testing
- QNN4EO training
- QNN4EO testing

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from CNN_Classifier import CNN_Classifier
from DatasetHandler import DatasetHandler
from QNN4EO import QNN4EO
import seaborn as sns

In [ ]:
datasetHandler = DatasetHandler('EuroSAT')

In [ ]:
epochs = 20
batch_size = 1

cnn_results = np.zeros((10, 10, 4))
qnn4eo_results = np.zeros((10, 10, 4))
classes_name = []

for i in range(10):
    class_1 = i
    classes_name.append(datasetHandler.classes[class_1].split('/')[-1])
    for j in range(10):
        if i!=j and j>i:
            class_2 = j

            print("\033[1m" + 'Load dataset:' + "\033[0m")
            print(datasetHandler.classes[class_1])
            print(datasetHandler.classes[class_2])

            cs = []
            cs.append(datasetHandler.classes[class_1].split('/')[-1])
            cs.append(datasetHandler.classes[class_2].split('/')[-1])

            imgs_path, imgs_label = datasetHandler.load_paths_labels(classes = [datasetHandler.classes[class_1], datasetHandler.classes[class_2]])
            train_images, train_labels, val_images, val_labels = datasetHandler.train_validation_split(imgs_path, imgs_label, split_factor = 0.2)

            img_shape = plt.imread(train_images[0]).shape
            n_classes = train_labels.shape[1]

            print("\033[1m" + 'Train CNN:' + "\033[0m")
            cnn = CNN_Classifier(img_shape = img_shape, n_classes = n_classes)
            train_generator = datasetHandler.cnn_data_loader(train_images, train_labels, batch_size = batch_size, img_shape = img_shape, n_classes = n_classes)
            val_generator = datasetHandler.cnn_data_loader(val_images, val_labels, batch_size = batch_size, img_shape = img_shape, n_classes = n_classes)
            history = cnn.train_model(epochs, batch_size, train_generator, val_generator, len(train_images), len(val_images))

            print("\033[1m" + 'Test CNN:' + "\033[0m")
            val_data_generator = datasetHandler.cnn_data_loader(val_images, val_labels, batch_size = len(val_images), img_shape = img_shape, n_classes = n_classes)
            cnn_results[i,j,...] = cnn.evaluate_model(val_data_generator, cs)
            
            cnn.model.save(os.path.join('trained_models', 'CNN-' + cs[0] + '-VS-' + cs[1] + '.h5'))

            print("\033[1m" + 'Train QNN4EO:' + "\033[0m")
            qnn4eo = QNN4EO()
            img_shape = plt.imread(train_images[0]).shape
            train_generator = datasetHandler.qcnn_data_loader(train_images, train_labels, batch_size = batch_size, img_shape = img_shape)
            loss_list = qnn4eo.train_model(epochs, train_generator, len(train_images))

            print("\033[1m" + 'Test QNN4EO:' + "\033[0m")
            val_data_gen = datasetHandler.qcnn_data_loader(val_images, val_labels, batch_size = batch_size, img_shape = img_shape)
            qnn4eo_results[i,j,...] = qnn4eo.evaluate_model(val_data_gen, len(val_images))

            torch.save(qnn4eo.state_dict(), os.path.join('trained_models', 'QNN4EO-' + cs[0] + '-VS-' + cs[1] + '.h5'))

        elif i==j:
            cnn_results[i,j] = np.nan
            qnn4eo_results[i,j] = np.nan
            
# Fill the lower triangular matrix with the upper triangluar one
cnn_results[...,0] = cnn_results[...,0] + cnn_results[...,0].T
cnn_results[...,1] = cnn_results[...,1] + cnn_results[...,1].T
cnn_results[...,2] = cnn_results[...,2] + cnn_results[...,2].T
cnn_results[...,3] = cnn_results[...,3] + cnn_results[...,3].T

qnn4eo_results[...,0] = qnn4eo_results[...,0] + qnn4eo_results[...,0].T
qnn4eo_results[...,1] = qnn4eo_results[...,1] + qnn4eo_results[...,1].T
qnn4eo_results[...,2] = qnn4eo_results[...,2] + qnn4eo_results[...,2].T
qnn4eo_results[...,3] = qnn4eo_results[...,3] + qnn4eo_results[...,3].T

The next cell displays the confusion matricies for Accuracy, Precision, Recall and F1 scores, both for the CNN and the QNN4EO.


In [ ]:
cmap = LinearSegmentedColormap.from_list(
    name='red-green', 
    colors=['red','green']
)

sns.set(font_scale=2)

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30, 10))

sns.heatmap(cnn_results[...,0], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[0]).set(title='CNN (Accuracy)')
sns.heatmap(qnn4eo_results[...,0], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[1]).set(title='QNN4EO (Accuracy)')
plt.show()

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30, 10))
sns.heatmap(cnn_results[...,1], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[0]).set(title='CNN (Precision)')
sns.heatmap(qnn4eo_results[...,1], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[1]).set(title='QNN4EO (Precision)')
plt.show()

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30, 10))
sns.heatmap(cnn_results[...,2], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[0]).set(title='CNN (Recall)')
sns.heatmap(qnn4eo_results[...,2], linewidth=0.5, fmt='.1f',  square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[1]).set(title='QNN4EO (Recall)')
plt.show()

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30, 10))
sns.heatmap(cnn_results[...,3], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[0]).set(title='CNN (F1)')
sns.heatmap(qnn4eo_results[...,3], linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = classes_name, yticklabels = classes_name, ax = axes[1]).set(title='QNN4EO (F1)')
plt.show()

In [ ]:
cnn_mean_accuracy = np.triu(cnn_results[...,0], k=1)
qnn4eo_mean_accuracy = np.triu(qnn4eo_results[...,0], k=1)

print('CNN overall accuracy: ',  cnn_mean_accuracy[cnn_mean_accuracy>0].mean())
print('QNN4EO overall accuracy: ',  qnn4eo_mean_accuracy[qnn4eo_mean_accuracy>0].mean())

# Paper Results

In [ ]:
from paper_results import import_results
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from CNN_Classifier import CNN_Classifier
from QNN4EO import QNN4EO


paper_cnn_results, paper_qnn4eo_results, paper_classes_name = import_results()

cmap = LinearSegmentedColormap.from_list(
    name='red-green', 
    colors=['red','green']
)

sns.set(font_scale=2)

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (30, 11))
sns.heatmap(paper_cnn_results, linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = paper_classes_name, yticklabels = paper_classes_name, ax = axes[0]).set(title='CNN (Accuracy)')
sns.heatmap(paper_qnn4eo_results, linewidth=0.5, fmt='.1f', square = True, cmap = cmap, 
            annot = True, xticklabels = paper_classes_name, yticklabels = paper_classes_name, ax = axes[1]).set(title='QNN4EO (Accuracy)')
plt.show()

In [ ]:
cnn_mean_accuracy = np.triu(paper_cnn_results, k=1)
qnn4eo_mean_accuracy = np.triu(paper_qnn4eo_results, k=1)


print('CNN overall accuracy: ',  cnn_mean_accuracy[cnn_mean_accuracy>0].mean())
print('QNN4EO overall accuracy: ',  qnn4eo_mean_accuracy[qnn4eo_mean_accuracy>0].mean())

## Test both

Load CNN

In [ ]:
from CNN_Classifier import CNN_Classifier
from tensorflow.keras.models import load_model

img_shape = plt.imread(train_images[0]).shape
n_classes = train_labels.shape[1]
cnn = CNN_Classifier(img_shape = img_shape, n_classes = n_classes)

cnn.model = load_model(os.path.join('trained_models', 'CNN-' + classes_name[0] + '-VS-' + classes_name[1] + '.h5'))

Load QNN4EO

In [ ]:
from QNN4EO import QNN4EO
import torch

qnn4eo = QNN4EO()

qnn4eo.load_state_dict(torch.load(os.path.join('trained_models', 'QNN4EO-' + classes_name[0] + '-VS-' + classes_name[1] + '.h5')))

Evaluate models

In [ ]:
qnn4eo.eval()

nrows = 2
ncols = 3

fig, axes = plt.subplots(nrows = nrows, ncols = ncols, figsize = (15, 10))

for x in range(nrows):
    for y in range(ncols):
        
        cnt = np.random.randint(0, high = len(val_images)-1, size=[1])[0]
        
        ground_truth = val_labels[cnt]
        
        img = plt.imread(val_images[cnt])/255.0
        
        cnn_input = np.zeros((1, 64, 64, 3))
        cnn_input[0, ...] = img
        
        cnn_pred = cnn.model.predict(cnn_input)
        
        qnn4eo_input = np.zeros((1, 3, 64, 64))
        qnn4eo_input[0, ...] = np.transpose(img)
        with torch.no_grad():
            qnn4eo_pred = qnn4eo(torch.Tensor(qnn4eo_input))
        
        
        
        #print(ground_truth, cnn_pred, qnn4eo_pred)
        axes[x,y].imshow(img)
        axes[x,y].set_title("Class " + str(classes_name[np.argmax(ground_truth)]) +": " +str(ground_truth)  + "\n" + 
                            "CNN Prediction: [%.2f, %.2f] \n QNN4EO Prediction: [%.2f, %.2f]" % (cnn_pred[0][0], cnn_pred[0][1], qnn4eo_pred[0].tolist()[0], qnn4eo_pred[0].tolist()[1]))
        
        axes[x,y].axis(False)

        
plt.show()